In [1]:
import pandas as pd
import mysql.connector

In [17]:
mydb = mysql.connector.connect(
    user='doadmin',
    password='00Cmib44qde23Wkj',
    host='db-mysql-sfo2-95612-do-user-9925891-0.b.db.ondigitalocean.com',
    port=25060,
    database='PrevWorks'
)

In [11]:
df = pd.read_csv('../data/processed/rates_days_away_from_work_by_occupation.csv')

In [12]:
df.drop([0], inplace = True)
df.drop([1], inplace = True)

In [13]:
df

,Unnamed: 0,Unnamed: 1,Total cases,Number of days away from work,Number of days away from work.1,Number of days away from work.2,Number of days away from work.3,Number of days away from work.4,Number of days away from work.5,Number of days away from work.6,Number of days away from work.7
2,All occupations,09-0000,0.00869,0.0011928179955416449,0.0009641749791718267,0.0015102962103983246,0.0009829595145346873,0.0009259210555943347,0.0005466104118349057,0.002567121996802594,7.826889734525229e-08
3,"Management, business, science, and arts occupa...",10-0000,0.0036899999999999997,0.0005910845522654299,0.0004551614341776668,0.0007464252586514449,0.00041599714591509097,0.0003725544059935783,0.0002096441312879058,0.0008991330717088833,1.974669996432394e-07
4,"Management, business, and financial occupations",10-1000,0.00234,0.00031271536193907634,0.00023913527677694074,0.0005886406812970849,0.00024450049132001314,0.00020311169341631185,0.00011726826072715363,0.0006346282345234196,4.598755322633476e-07
5,Management occupations,11-0000,0.0035700000000000003,0.0004702650602409639,0.00035269879518072295,0.0009261927710843374,0.0003871084337349398,0.00027384337349397594,0.00017634939759036147,0.0009835421686746988,8.602409638554217e-07
6,Top executives,11-1000,0.00145,0.00025375,0.00013694444444444445,0.0004551388888888889,0.00018125,0.00011277777777777778,7.652777777777778e-05,0.00022958333333333332,2.013888888888889e-06
...,...,...,...,...,...,...,...,...,...,...,...
772,Stockers and order fillers,53-7065,0.01763,0.002143406352683461,0.001976053304125593,0.002890058415480102,0.0020404198612632347,0.0021627163198247533,0.0010749215041986125,0.005342424242424242,6.436655713764147e-06
773,Pumping station operators,53-7070,0.0128,0.0015058823529411766,0.0,0.0,0.005270588235294118,0.0,0.003388235294117647,0.0022588235294117645,0.00037647058823529414
774,"Pump operators, except wellhead pumpers",53-7072,0.03021,0.0,0.0,0.0,0.014098000000000001,0.0,0.009063,0.0060420000000000005,0.0024168
775,Wellhead pumpers,53-7073,0.00315,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.875e-05


In [14]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Total cases',
       'Number of days away from work', 'Number of days away from work.1',
       'Number of days away from work.2', 'Number of days away from work.3',
       'Number of days away from work.4', 'Number of days away from work.5',
       'Number of days away from work.6', 'Number of days away from work.7'],
      dtype='object')

In [15]:
df.columns = ['Unnamed: 0', 'Unnamed: 1', 'Total cases',
       '1', '2',
       '4', '8',
       '15', '25',
       '31', 'Number of days away from work.7']

In [8]:
parts = df.columns[2:]

In [9]:
parts = {'Total cases': 1, 'Head': 2, 'Eye': 3, 'Neck': 4, 'Trunk': 5, 'Back': 6,
       'Upper extremities': 7, 'Shoulder': 8, 'Arm': 9, 'Hand': 10, 'Wrist': 11,
       'Lower extremities': 12, 'Knee': 13, 'Ankle': 14, 'Foot': 15, 'Body systems': 16,
       'Multiple body parts': 17, 'All other body parts': 18}

In [ ]:
mycursor = mydb.cursor()
for b in parts:
    mycursor.execute(
        "INSERT INTO bodyParts (name) VALUES ('{}')".format(b),
    )
mydb.commit()

In [20]:
mycursor = mydb.cursor()
flag = 0
for ind, row in df.iterrows():
    for col in df.columns[3:-1]:
        print(row['Unnamed: 0'] + ' ' + col)
        mycursor.execute(
            "SELECT occupationId FROM occupations WHERE occupationName = '{}'".format(row['Unnamed: 0'].replace("'", ''))
        )
        user = mycursor.fetchone()
        if user == None:
            mycursor.execute(
                "INSERT INTO occupations (occupationName) VALUES ('{}')".format(row['Unnamed: 0'].replace("'", ''))
            )
            mycursor.execute(
                "SELECT occupationId FROM occupations WHERE occupationName = '{}'".format(row['Unnamed: 0'].replace("'", ''))
            )

            user = mycursor.fetchone()
        mycursor.execute(
            "INSERT INTO occupationDaysProbabilities (occupationId, probability, avgDays) VALUES ({}, {}, {})".format(user[0],  float(row[col]), int(col)),
        )
        mydb.commit()

All occupations 1
All occupations 2
All occupations 4
All occupations 8
All occupations 15
All occupations 25
All occupations 31
Management, business, science, and arts occupations 1
Management, business, science, and arts occupations 2
Management, business, science, and arts occupations 4
Management, business, science, and arts occupations 8
Management, business, science, and arts occupations 15
Management, business, science, and arts occupations 25
Management, business, science, and arts occupations 31
Management, business, and financial occupations 1
Management, business, and financial occupations 2
Management, business, and financial occupations 4
Management, business, and financial occupations 8
Management, business, and financial occupations 15
Management, business, and financial occupations 25
Management, business, and financial occupations 31
Management occupations 1
Management occupations 2
Management occupations 4
Management occupations 8
Management occupations 15
Management 

Property, real estate, and community association managers 1
Property, real estate, and community association managers 2
Property, real estate, and community association managers 4
Property, real estate, and community association managers 8
Property, real estate, and community association managers 15
Property, real estate, and community association managers 25
Property, real estate, and community association managers 31
Social and community service managers 1
Social and community service managers 2
Social and community service managers 4
Social and community service managers 8
Social and community service managers 15
Social and community service managers 25
Social and community service managers 31
Emergency management directors 1
Emergency management directors 2
Emergency management directors 4
Emergency management directors 8
Emergency management directors 15
Emergency management directors 25
Emergency management directors 31
Miscellaneous managers 1
Miscellaneous managers 2
Miscellane

Credit counselors 4
Credit counselors 8
Credit counselors 15
Credit counselors 25
Credit counselors 31
Loan officers 1
Loan officers 2
Loan officers 4
Loan officers 8
Loan officers 15
Loan officers 25
Loan officers 31
Tax examiners, collectors and preparers, and revenue agents 1
Tax examiners, collectors and preparers, and revenue agents 2
Tax examiners, collectors and preparers, and revenue agents 4
Tax examiners, collectors and preparers, and revenue agents 8
Tax examiners, collectors and preparers, and revenue agents 15
Tax examiners, collectors and preparers, and revenue agents 25
Tax examiners, collectors and preparers, and revenue agents 31
Tax preparers 1
Tax preparers 2
Tax preparers 4
Tax preparers 8
Tax preparers 15
Tax preparers 25
Tax preparers 31
Computer, engineering, and science occupations 1
Computer, engineering, and science occupations 2
Computer, engineering, and science occupations 4
Computer, engineering, and science occupations 8
Computer, engineering, and science

Mechanical engineers 15
Mechanical engineers 25
Mechanical engineers 31
Mining and geological engineers, including mining safety engineers 1
Mining and geological engineers, including mining safety engineers 2
Mining and geological engineers, including mining safety engineers 4
Mining and geological engineers, including mining safety engineers 8
Mining and geological engineers, including mining safety engineers 15
Mining and geological engineers, including mining safety engineers 25
Mining and geological engineers, including mining safety engineers 31
Drafters, engineering technicians, and mapping technicians 1
Drafters, engineering technicians, and mapping technicians 2
Drafters, engineering technicians, and mapping technicians 4
Drafters, engineering technicians, and mapping technicians 8
Drafters, engineering technicians, and mapping technicians 15
Drafters, engineering technicians, and mapping technicians 25
Drafters, engineering technicians, and mapping technicians 31
Drafters 1
D

Agricultural and food science technicians 2
Agricultural and food science technicians 4
Agricultural and food science technicians 8
Agricultural and food science technicians 15
Agricultural and food science technicians 25
Agricultural and food science technicians 31
Biological technicians 1
Biological technicians 2
Biological technicians 4
Biological technicians 8
Biological technicians 15
Biological technicians 25
Biological technicians 31
Chemical technicians 1
Chemical technicians 2
Chemical technicians 4
Chemical technicians 8
Chemical technicians 15
Chemical technicians 25
Chemical technicians 31
Environmental science and geoscience technicians 1
Environmental science and geoscience technicians 2
Environmental science and geoscience technicians 4
Environmental science and geoscience technicians 8
Environmental science and geoscience technicians 15
Environmental science and geoscience technicians 25
Environmental science and geoscience technicians 31
Environmental science and prote

...

Model makers and patternmakers, metal and plastic 25
Model makers and patternmakers, metal and plastic 31
Model makers, metal and plastic 1
Model makers, metal and plastic 2
Model makers, metal and plastic 4
Model makers, metal and plastic 8
Model makers, metal and plastic 15
Model makers, metal and plastic 25
Model makers, metal and plastic 31
Molders and molding machine setters, operators, and tenders, metal and plastic 1
Molders and molding machine setters, operators, and tenders, metal and plastic 2
Molders and molding machine setters, operators, and tenders, metal and plastic 4
Molders and molding machine setters, operators, and tenders, metal and plastic 8
Molders and molding machine setters, operators, and tenders, metal and plastic 15
Molders and molding machine setters, operators, and tenders, metal and plastic 25
Molders and molding machine setters, operators, and tenders, metal and plastic 31
Foundry mold and coremakers 1
Foundry mold and coremakers 2
Foundry mold and corema

Tailors, dressmakers, and custom sewers 15
Tailors, dressmakers, and custom sewers 25
Tailors, dressmakers, and custom sewers 31
Textile machine setters, operators, and tenders 1
Textile machine setters, operators, and tenders 2
Textile machine setters, operators, and tenders 4
Textile machine setters, operators, and tenders 8
Textile machine setters, operators, and tenders 15
Textile machine setters, operators, and tenders 25
Textile machine setters, operators, and tenders 31
Textile bleaching and dyeing machine operators and tenders 1
Textile bleaching and dyeing machine operators and tenders 2
Textile bleaching and dyeing machine operators and tenders 4
Textile bleaching and dyeing machine operators and tenders 8
Textile bleaching and dyeing machine operators and tenders 15
Textile bleaching and dyeing machine operators and tenders 25
Textile bleaching and dyeing machine operators and tenders 31
Textile cutting machine setters, operators, and tenders 1
Textile cutting machine setter

Chemical processing machine setters, operators, and tenders 8
Chemical processing machine setters, operators, and tenders 15
Chemical processing machine setters, operators, and tenders 25
Chemical processing machine setters, operators, and tenders 31
Chemical equipment operators and tenders 1
Chemical equipment operators and tenders 2
Chemical equipment operators and tenders 4
Chemical equipment operators and tenders 8
Chemical equipment operators and tenders 15
Chemical equipment operators and tenders 25
Chemical equipment operators and tenders 31
Separating, filtering, clarifying, precipitating, and still machine setters, operators, and tenders 1
Separating, filtering, clarifying, precipitating, and still machine setters, operators, and tenders 2
Separating, filtering, clarifying, precipitating, and still machine setters, operators, and tenders 4
Separating, filtering, clarifying, precipitating, and still machine setters, operators, and tenders 8
Separating, filtering, clarifying, pr

Photographic process workers and processing machine operators 25
Photographic process workers and processing machine operators 31
Computer numerically controlled tool operators and programmers 1
Computer numerically controlled tool operators and programmers 2
Computer numerically controlled tool operators and programmers 4
Computer numerically controlled tool operators and programmers 8
Computer numerically controlled tool operators and programmers 15
Computer numerically controlled tool operators and programmers 25
Computer numerically controlled tool operators and programmers 31
Miscellaneous production workers 1
Miscellaneous production workers 2
Miscellaneous production workers 4
Miscellaneous production workers 8
Miscellaneous production workers 15
Miscellaneous production workers 25
Miscellaneous production workers 31
Adhesive bonding machine operators and tenders 1
Adhesive bonding machine operators and tenders 2
Adhesive bonding machine operators and tenders 4
Adhesive bonding 

Railroad brake, signal, and switch operators and locomotive firers 8
Railroad brake, signal, and switch operators and locomotive firers 15
Railroad brake, signal, and switch operators and locomotive firers 25
Railroad brake, signal, and switch operators and locomotive firers 31
Railroad conductors and yardmasters 1
Railroad conductors and yardmasters 2
Railroad conductors and yardmasters 4
Railroad conductors and yardmasters 8
Railroad conductors and yardmasters 15
Railroad conductors and yardmasters 25
Railroad conductors and yardmasters 31
Water transportation workers 1
Water transportation workers 2
Water transportation workers 4
Water transportation workers 8
Water transportation workers 15
Water transportation workers 25
Water transportation workers 31
Sailors and marine oilers 1
Sailors and marine oilers 2
Sailors and marine oilers 4
Sailors and marine oilers 8
Sailors and marine oilers 15
Sailors and marine oilers 25
Sailors and marine oilers 31
Ship and boat captains and operat

In [19]:
mydb.commit()